<a href="https://colab.research.google.com/github/victorviro/Deep_learning_python/blob/master/TensorFlow%E2%80%99s_autodifferentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow's autodifferentiation

# Table of contents

1. [Introduction](#1)
2. [Manual differentiation](#2)
3. [Finite difference approximation](#3)
4. [Forward-Mode autodiff](#4)
5. [Reverse-Mode autodiff](#5)
6. [Reverse-Mode autodiff with TensorFlow](#6)
7. [References and further reading](#7)


# Introduction <a name="1"></a>

In this notebook, we explain briefly how TensorFlow’s [auto-differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) (autodiff) feature works, and how it compares to other solutions.

Suppose we define a function $f(x,y) = x^2y+y+2$, and we need its **partial derivatives** $\frac{\partial f}{\partial x}$ and $\frac{\partial f}{\partial y}$, typically **to perform Gradient Descent** (or some other optimization algorithm). Our main **options** are manual differentiation, finite difference approximation, forward-mode autodiff, and reverse-mode autodiff. **TensorFlow implements reverse-mode autodiff**, but to understand it, it’s useful to look at the other options first. So let’s go through each of them, starting with manual differentiation.

# Manual differentiation <a name="2"></a>

The first approach to compute derivatives is to pick up a pencil and a paper, and **use our calculus knowledge to derive the appropriate equation**:

$$\frac{\partial f}{\partial x}=2xy$$
$$\frac{\partial f}{\partial y}=x^2+1$$

This approach can become very **tedious for more complex functions** 😓, and we run the risk of **making mistakes**.

# Finite difference approximation <a name="3"></a>

The [derivative](https://en.wikipedia.org/wiki/Derivative) $h^{\prime}(x_0)$ of a function $h(x)$ at a point $x_0$ is the slope of the function at that point. More precisely, the derivative is defined as the limit of the slope of a straight line going through this point $x_0$ and another point $x$ on the function, as $x$ gets infinitely close to $x_0$:

$$h^{\prime}(x_0) = \lim_{x \to x_0} \frac{h(x)-h(x_0)}{x-x_0} = \lim_{\epsilon \to 0} \frac{h(x_0+\epsilon)-h(x_0)}{\epsilon}$$

So, if we want to calculate the partial derivatives of $f(x, y)$ with regard to $x$ and $y$ at a point $(x,y)$ , we could compute

$$\nabla f(x,y)=(\frac{\partial f(x,y)}{\partial x},\frac{\partial f(x,y)}{\partial x}) \approx (\frac{f(x+\epsilon,y)-f(x,y)}{\epsilon}, \frac{f(x,y+\epsilon)-f(x,y)}{\epsilon})$$

This type of numerical approximation of the derivative is called a **finite difference approximation**, and this specific equation is called *Newton’s difference quotient*. That’s exactly what the following code does:

In [2]:
def f(x, y):
    return x**2*y + y + 2

def compute_gradient(x, y, eps):
    gradient = (f(x + eps, y) - f(x, y)) / eps, (f(x, y + eps) - f(x, y)) / eps
    return gradient

# Calculate the derivatives of f at point (3,4)
print(compute_gradient(3, 4, 0.00001))

(24.000039999805264, 10.000000000331966)


Unfortunately, the result is **imprecise** (and it gets worse for more complicated functions). The correct results are respectively 24 and 10.

Notice that to compute both partial derivatives, we have to call `f()` at least three times (we called it four times in the preceding code, but it could be optimized). If there were 1000 parameters, we would need to call `f()` at least 1001 times. When we are **dealing with large neural networks**, this makes finite difference approximation way too **inefficient**.

So far, we have considered two ways to compute gradients: using manual differentiation and using finite difference approximation. Unfortunately, both were fatally flawed to train a large-scale neural network. So let’s turn to autodiff, starting with forward mode.

# Forward-Mode autodiff <a name="4"></a>

The next figure shows how forward-mode autodiff works on an even simpler function, $g(x,y)=5+xy$. The graph for that function is represented on the left. After forward-mode autodiff, we get the graph on the right, which represents the partial derivative $\frac{\partial g}{\partial x}= 0 + (0x + y) = y$ (we could similarly obtain the partial derivative with regard to $y$).

![texto alternativo](https://i.ibb.co/2YcfWhn/fordward-mode-autodiff.png)

The algorithm will go through the computation graph from the inputs to the outputs (hence the name “forward mode”). 

- It starts by getting the partial derivatives of the leaf nodes. The constant node (5) returns the constant 0, since the derivative of a constant is always 0. The variable $x$ returns the constant 1 since $\frac{\partial x}{\partial x} = 1$, and the variable $y$ returns the constant 0 since $\frac{\partial y}{\partial x} = 0$.

- Now we have all we need to move up the graph to the multiplication node in function $g$. Calculus theory tells us that the derivative of the product of two functions $u$ and $v$ is $\frac{\partial (uv)}{\partial x}=\frac{\partial v}{\partial x} u + \frac{\partial u}{\partial x} v$. We can therefore construct a large part of the graph on the right, representing $0x + y$.

- Finally, we can go up to the addition node in function $g$. As the derivative of a sum of functions is the sum of these function's derivatives, we just need to create an addition node and connect it to the parts of the graph we have already computed. We get the correct partial derivative: 
$\frac{\partial g}{\partial x} = 0 + (0x + y)$.

However, this equation can be simplified (a lot). A few pruning steps can be applied to the computation graph to get rid of all unnecessary operations, and we get a much smaller graph with just one node: $\frac{\partial g}{\partial x} =y$. In this case, simplification is fairly easy, but **for a more complex function, forward-mode autodiff can produce a huge graph** that may be tough to simplify and lead to **suboptimal performance**.

Note that we started with a computation graph, and forward-mode autodiff produced another computation graph. This is known as **symbolic differentiation**, and it has two nice features:

1. Once the computation graph of the derivative has been produced, we can use it as many times as we want to compute the derivatives of the given function for any value of $x$ and $y$.

2. We can run forward-mode autodiff again on the resulting graph to get second-order derivatives if we ever need to (i.e., derivatives of derivatives). We could even compute third-order derivatives, and so on.

It is also **possible** to **run forward-mode autodiff** without constructing a graph (i.e., **numerically, not symbolically**), just by computing intermediate results on the fly.

So **forward-mode autodiff** is much **more accurate than finite difference approximation, but it suffers from the same major flaw**, at least when there are many inputs and few outputs (as is the case when dealing with neural networks): if there were 1000 parameters, it would require 1000 passes through the graph to compute all the partial derivatives. This is where reverse-mode autodiff shines: it can compute all of them in just two passes through the graph. Let’s see how.

# Reverse-Mode autodiff <a name="5"></a>

Reverse-mode autodiff is the solution **implemented by TensorFlow**. It first goes through the graph in the forward direction (i.e., from the inputs to the output) to compute the value of each node. Then it does a second pass, this time in the reverse direction (i.e., from the output to the inputs), to compute all the partial derivatives. The name "reverse mode" comes from this second pass through the graph, where gradients flow in the reverse direction. 

Figure D-3 represents the second pass to calculate all the partial derivatives of the function $f(x,y) = x^2y+y+2$ at the point $(x,y)=(3,4)$. During the first pass, all the node values were computed, starting from $x = 3$ and $y = 4$. We can see those values at the bottom right of each node (e.g., $x^2 = 9$). The nodes are labeled $n_1$ to $n_7$ for clarity. The output node is $n_7$: $f(3, 4) = n_7 = 42$.

![texto alternativo](https://i.ibb.co/v4LQtnp/reverse-mode-autodiff.png)

The idea is to gradually go down the graph, computing the partial derivative of $f(x, y)$ with regard to each consecutive node, until we reach the variable nodes. For this, reverse-mode autodiff relies heavily on the [chain rule](https://en.wikipedia.org/wiki/Chain_rule), shown in the next equation.

$$\frac{\partial f}{\partial x}=\frac{\partial f}{\partial n_i}\times\frac{\partial n_i}{\partial x}$$

- Since $n_7$ is the output node, $f = n_7$ so $\frac{\partial f}{\partial n_7}=1$.

- Let’s continue down the graph to $n_5$: how much does $f$ vary when $n_5$ varies? The answer is $\frac{\partial f}{\partial n_5}=\frac{\partial f}{\partial n_7}\times\frac{\partial n_7}{\partial n_5}$. We already know that $\frac{\partial f}{\partial n_7}=1$, so all we need is $\frac{\partial n_7}{\partial n_5}$. Since $n_7$ simply performs the sum $n_5 + n_6$, we find that $\frac{\partial n_7}{\partial n_5}$=1, so $\frac{\partial f}{\partial n_5}=1\times 1=1$.

- Now we can proceed to node $n_4$: how much does $f$ vary when $n_4$ varies? The answer is $\frac{\partial f}{\partial n_4}=\frac{\partial f}{\partial n_5}\times\frac{\partial n_5}{\partial n_4}$. Since $n_5 = n_4 \times n_2$, we find that $\frac{\partial n_5}{\partial n_4}=n_2$, so $\frac{\partial f}{\partial n_4}=1 \times n_2=4$.

- The process continues until we reach the bottom of the graph. At that point we will have calculated all the partial derivatives of $f(x, y)$ at the point $x = 3$ and $y = 4$. In this example, we find $\frac{\partial f}{\partial x}=24$ and $\frac{\partial y}{\partial y}=10$.

**Reverse-mode autodiff** is a very powerful and accurate technique, especially when there are many inputs and few outputs, since it **requires only one forward pass plus one reverse pass per output to compute all the partial derivatives for all outputs with regard to all the inputs**. When training neural networks, we generally want to minimize the loss, so there is a single output (the loss), and hence only two passes through the graph are needed to compute the gradients. Reverse-mode autodiff can also handle functions that are not entirely differentiable, as long as we ask it to compute the partial derivatives at points that are differentiable.

In Figure D-3, the numerical results are computed on the fly, at each node. However, that’s not exactly what **TensorFlow** does: instead, it **creates a new computation graph**. In other words, it implements **symbolic reverse-mode autodiff**. This way, the **computation graph** to compute the gradients of the loss with regard to all the parameters in the neural network **only needs to be generated once**, and then it can be executed over and over again, whenever the optimizer needs to compute the gradients. Moreover, this makes it possible to compute higher-order derivatives if needed.

# Reverse-Mode autodiff with TensorFlow <a name="6"></a>

TensorFlow provides the [`tf.GradientTape`](https://www.tensorflow.org/api_docs/python/tf/GradientTape) API for automatic differentiation. TensorFlow "records" relevant operations executed inside the context of a `tf.GradientTape` onto a "tape". TensorFlow then uses that tape to compute the gradients of a "recorded" computation using reverse-mode differentiation.

In [ ]:
import tensorflow as tf

x = tf.Variable(3.)
y = tf.Variable(4.)

def f(x, y):
    return x**2*y + y + 2

with tf.GradientTape() as tape:
    #f = x**2*y + y + 2
    z = f(x, y)

gradients = tape.gradient(z, [x, y])

We first define two variables `x` and `y`. Then we create a `tf.GradientTape` context that will automatically record every operation that involves a variable, and finally, we ask this tape to compute the gradients of the result `z` with regards to both variables `[x, y]` by calling its [`gradient`](https://www.tensorflow.org/api_docs/python/tf/GradientTape#gradient) method. Let’s take a look at the gradients that TensorFlow computed:

In [ ]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=24.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

The result is accurate (the precision is only limited by the floating-point errors), and the `gradient()` method only goes through the recorded computations once (in reverse order), no matter how many variables there are, so it is incredibly efficient. The above example uses scalars, but `tf.GradientTape` works as easily on any tensor.

**Note**: We just put the strict minimum inside the `tf.GradientTape()` block, to save memory. Alternatively, we can pause recording by creating a `with tape.stop_recording()` block inside the `tf.GradientTape()` block.

The tape is automatically erased immediately after we call its `gradient()` method, so we will get an exception if we try to call `gradient()` twice:

In [ ]:
# gradients = tape.gradient(z, [x, y])

If we need to call `gradient()` more than once, we must make the tape persistent, and delete it when we are done with it to free resources:

In [ ]:
with tf.GradientTape(persistent=True) as tape:
    z = f(x, y)
    
dz_dx = tape.gradient(z, w1) # => tensor 24.0
dz_dy = tape.gradient(z, w2) # => tensor 10.0
del tape

By default, the tape will only track operations involving variables, so if we try to compute the gradient of `z` with regards to anything else than a variable, the result will be `None`:

In [ ]:
c1, c2 = tf.constant(3.), tf.constant(4.)
with tf.GradientTape() as tape:
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])
gradients

[None, None]

However, we can force the tape to watch any tensors we like, to record every operation that involves them. We can then compute gradients with regards to these tensors, as if they were variables:

In [ ]:
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=24.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

This can be useful in some cases, for example, if we want to implement a regularization loss that penalizes activations that vary a lot when the inputs vary little: the loss will be based on the gradient of the activations with regards to the inputs. Since the inputs are not variables, we would need to tell the tape to watch them.

If we compute the gradient of a list of tensors (e.g., `[z1, z2, z3]` ) with regards to some variables (e.g., `[x, y]` ), TensorFlow actually efficiently computes the sum of the gradients of these tensors (i.e., gradient `(z1, [x, y])`, plus gradient `(z2, [x, y])`, plus gradient `(z3, [x, y])`). 

In [ ]:
with tf.GradientTape() as tape:
    z1 = f(x, y + 2.)
    z2 = f(x, y + 5.)
    z3 = f(x, y + 7.)

tape.gradient([z1, z2, z3], [x, y])

[<tf.Tensor: shape=(), dtype=float32, numpy=156.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=30.0>]

Due to the way reverse-mode autodiff works, it is not possible to compute the individual gradients (`z1`, `z2` and `z3`) without actually calling `gradient()` multiple times (once for `z1`, once for `z2` and once for `z3`),
which requires making the tape persistent (and deleting it afterward).

In [ ]:
with tf.GradientTape(persistent=True) as tape:
    z1 = f(x, y + 2.)
    z2 = f(x, y + 5.)
    z3 = f(x, y + 7.)

for z in [z1, z2, z3]:
    print(tape.gradient(z, [x, y]))
del tape

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]
[<tf.Tensor: shape=(), dtype=float32, numpy=54.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]
[<tf.Tensor: shape=(), dtype=float32, numpy=66.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]


In some rare cases, we may want to stop gradients from backpropagating through some part of our neural network. To do this, we must use the `tf.stop_gradient()`
function: it just returns its inputs during the forward pass (like `tf.identity()`), but it does not let gradients through during backpropagation (it acts like a constant). For example:

In [ ]:
def f(x, y):
    return 3 * x ** 2 + tf.stop_gradient(2 * x * y)

with tf.GradientTape() as tape:
    z = f(x, y)

tape.gradient(z, [x, y])

[<tf.Tensor: shape=(), dtype=float32, numpy=18.0>, None]

More information about how the `tf.GradientTape` API works is available in the [TensorFlow documentation](https://www.tensorflow.org/guide/autodiff).

# References and further reading <a name="7"></a>

- [TensorFlow: Introduction to gradients and automatic differentiation](https://www.tensorflow.org/guide/autodiff)

- [TensorFlow: Custom gradient](https://www.tensorflow.org/guide/eager#custom_gradients)

- [TensorFlow: Advanced automatic differentiation](https://www.tensorflow.org/guide/advanced_autodiff)

- [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

